In [1]:
import pandas as pd
import numpy as np

# The pandas and numpy library are imported here

In [49]:
laptops = pd.read_csv("laptops.csv",encoding= "Latin-1")
laptops.head()

,Manufacturer,Model Name,Category,Screen Size,Screen,CPU,RAM,Storage,GPU,Operating System,Operating System Version,Weight,Price (Euros)
0,Apple,MacBook Pro,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,NaN,1.37kg,"1339,69"
1,Apple,Macbook Air,Ultrabook,"13.3""",1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,NaN,1.34kg,"898,94"
2,HP,250 G6,Notebook,"15.6""",Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,NaN,1.86kg,"575,00"
3,Apple,MacBook Pro,Ultrabook,"15.4""",IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,NaN,1.83kg,"2537,45"
4,Apple,MacBook Pro,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,NaN,1.37kg,"1803,60"


## The "laptops.csv" file is read into a dataframe named "laptops"

In [3]:
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
Manufacturer                1303 non-null object
Model Name                  1303 non-null object
Category                    1303 non-null object
Screen Size                 1303 non-null object
Screen                      1303 non-null object
CPU                         1303 non-null object
RAM                         1303 non-null object
 Storage                    1303 non-null object
GPU                         1303 non-null object
Operating System            1303 non-null object
Operating System Version    1133 non-null object
Weight                      1303 non-null object
Price (Euros)               1303 non-null object
dtypes: object(13)
memory usage: 132.4+ KB


## Laptops.info() helps us check the type of data we are working with and also shows the number of entries for each column.

In [4]:
laptops.columns

Index(['Manufacturer', 'Model Name', 'Category', 'Screen Size', 'Screen',
       'CPU', 'RAM', ' Storage', 'GPU', 'Operating System',
       'Operating System Version', 'Weight', 'Price (Euros)'],
      dtype='object')

In [50]:
def clean_col(col):
    col = col.strip()
    col = col.replace("Operating System","os")
    col = col.replace(" ","_")
    col = col.replace("(","")
    col = col.replace(")","")
    col = col.lower()
    return col
laptops.columns = [clean_col(c) for c in laptops.columns]
print(laptops.columns)

Index(['manufacturer', 'model_name', 'category', 'screen_size', 'screen',
       'cpu', 'ram', 'storage', 'gpu', 'os', 'os_version', 'weight',
       'price_euros'],
      dtype='object')


## Here i cleaned the column names and we can see the changes in the column names in this cell in comparison the cell above.

In [51]:
laptops["screen_size"] = laptops["screen_size"].str.replace('"','').astype(float)
laptops.rename({"screen_size": "screen_size_inches"}, axis=1, inplace=True)

laptops["ram"] = laptops["ram"].str.replace('GB','').astype(int)
laptops.rename({"ram": "ram_gb"}, axis=1, inplace=True)

laptops["weight"] = laptops["weight"].str.replace("kgs","").str.replace("kg","").astype(float)
laptops.rename({"weight": "weight_kg"}, axis=1, inplace=True)

laptops["price_euros"] = laptops["price_euros"].str.replace(",",".").astype(float)

In [10]:
laptops.iloc[:3,[3,6,11,12]]

,screen_size_inches,ram_gb,weight_kg,price_euros
0,13.3,8,1.37,1339.69
1,13.3,8,1.34,898.94
2,15.6,8,1.86,575.00


## As we can see from above cell i cleaned the "screen_size","ram","weight","price_euros" columns by str.replace and converted it to numerical using astype.

In [52]:
screen_res = laptops["screen"].str.rsplit(n=1, expand=True)
screen_res.columns = ["A", "B"]
screen_res.loc[screen_res["B"].isnull(), "B"] = screen_res["A"]
laptops["screen_resolution"] = screen_res["B"]

## As we can see from the cell above,the first code first splitted each entry of "screen" column and then in the second line,the column names were renamed,the third line accounted for imputing the null values in column "B" from column "A" and finally it is renamed in the last code and added to laptops dataframe.  

In [12]:
laptops["cpu"].head(3)

0          Intel Core i5 2.3GHz
1          Intel Core i5 1.8GHz
2    Intel Core i5 7200U 2.5GHz
3          Intel Core i7 2.7GHz
4          Intel Core i5 3.1GHz
Name: cpu, dtype: object

In [53]:
laptops["cpu_speed_ghz"] = (laptops["cpu"]
                            .str.replace("GHz","")
                            .str.rsplit(n=1,expand=True)
                            .iloc[:,1]
                            .astype(float)
                            )

In [14]:
laptops["cpu_speed_ghz"].head(3)

0    2.3
1    1.8
2    2.5
Name: cpu_speed_ghz, dtype: float64

## In the code above we cleaned the column by replacing "GHz" and then splitting backwards by 1 partition using str.rsplit,then took the second column using iloc and finally converted to float datatype

In [54]:
laptops["gpu_manufacturer"] = (laptops["gpu"].str.split(n=1,expand=True).iloc[:,0])
laptops["cpu_manufacturer"] = (laptops["cpu"].str.split(n=1,expand=True).iloc[:,0]) 

## The name of the companies were extracted from the "gpu" and the "cpu" columns using str.split and taking the first column

In [17]:
laptops.iloc[:3,13:17]

,screen_resolution,cpu_speed_ghz,gpu_manufacturer,cpu_manufacturer
0,2560x1600,2.3,Intel,Intel
1,1440x900,1.8,Intel,Intel
2,1920x1080,2.5,Intel,Intel


## The new columns created are shown above

## CORRECTING BAD VALUES check for columns having "Mac OS " and rewrite as "macOS"

In [55]:
mapping_dict = {
    'Android': 'Android',
    'Chrome OS': 'Chrome OS',
    'Linux': 'Linux',
    'Mac OS': 'macOS',
    'No OS': 'No OS',
    'Windows': 'Windows',
    'macOS': 'macOS'}

In [56]:
laptops["os"] = laptops["os"].map(mapping_dict)

In [44]:
laptops["os"].value_counts()

Windows      1125
No OS          66
Linux          62
Chrome OS      27
macOS          21
Android         2
Name: os, dtype: int64

## The mapping of the "os" column is done to clear the confusion between "Mac OS" and "macOS"

In [57]:
laptops.loc[laptops["os"] == "macOS", "os_version"] = "X"
laptops.loc[laptops["os"] == "No OS", "os_version"] = "Version Unknown"

## As there were some missing values in "os_version", we imput it with "X"

In [46]:
laptops["storage"].unique()

array(['128GB SSD', '128GB Flash Storage', '256GB SSD', '512GB SSD',
       '500GB HDD', '256GB Flash Storage', '1TB HDD',
       '32GB Flash Storage', '128GB SSD +  1TB HDD',
       '256GB SSD +  256GB SSD', '64GB Flash Storage',
       '256GB SSD +  1TB HDD', '256GB SSD +  2TB HDD', '32GB SSD',
       '2TB HDD', '64GB SSD', '1TB Hybrid', '512GB SSD +  1TB HDD',
       '1TB SSD', '256GB SSD +  500GB HDD', '128GB SSD +  2TB HDD',
       '512GB SSD +  512GB SSD', '16GB SSD', '16GB Flash Storage',
       '512GB SSD +  256GB SSD', '512GB SSD +  2TB HDD',
       '64GB Flash Storage +  1TB HDD', '1GB SSD', '1TB HDD +  1TB HDD',
       '32GB HDD', '1TB SSD +  1TB HDD', '512GB Flash Storage',
       '128GB HDD', '240GB SSD', '8GB SSD', '508GB Hybrid',
       '512GB SSD +  1TB Hybrid', '256GB SSD +  1TB Hybrid'], dtype=object)

In [58]:
a = laptops["storage"].str.replace("GB","").str.replace("TB","000")
a = a.str.split("+",expand = True)
#spliiting the a[0] part
b = a[0].str.split(n=1,expand = True)
b[0] = b[0].astype(float)
b[1] = b[1].str.strip()
laptops[["storage_1_capacity_gb","storage_1_type"]] = b[[0,1]]
#splitting the a[1] part
c = a[1].str.split(n=1,expand = True)
c[0] = c[0].astype(float)
c[1] = c[1].str.strip()
laptops[["storage_2_capacity_gb","storage_2_type"]] = c[[0,1]]

del a
del b
del c
laptops = laptops.drop('storage',axis = 1)


## Here i cleaned up the storage to convert it into usable features

In [59]:
cols = ['manufacturer', 'model_name', 'category', 'screen_size_inches',
        'screen','screen_resolution', 'cpu', 'cpu_manufacturer',  'cpu_speed_ghz', 'ram_gb',
        'storage_1_type', 'storage_1_capacity_gb', 'storage_2_type',
        'storage_2_capacity_gb', 'gpu', 'gpu_manufacturer', 'os',
        'os_version', 'weight_kg', 'price_euros']

laptops = laptops[cols]

In [60]:
laptops.to_csv('laptops_cleaned.csv',index=False)
laptops_cleaned = pd.read_csv('laptops_cleaned.csv')

In [61]:
laptops_cleaned.head(3)

,manufacturer,model_name,category,screen_size_inches,screen,screen_resolution,cpu,cpu_manufacturer,cpu_speed_ghz,ram_gb,storage_1_type,storage_1_capacity_gb,storage_2_type,storage_2_capacity_gb,gpu,gpu_manufacturer,os,os_version,weight_kg,price_euros
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,2560x1600,Intel Core i5 2.3GHz,Intel,2.3,8,SSD,128.0,NaN,NaN,Intel Iris Plus Graphics 640,Intel,macOS,X,1.37,1339.69
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,1440x900,Intel Core i5 1.8GHz,Intel,1.8,8,Flash Storage,128.0,NaN,NaN,Intel HD Graphics 6000,Intel,macOS,X,1.34,898.94
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,1920x1080,Intel Core i5 7200U 2.5GHz,Intel,2.5,8,SSD,256.0,NaN,NaN,Intel HD Graphics 620,Intel,No OS,Version Unknown,1.86,575.00


## In the above cells,the oredered the sequence of the column names and produced a new csv file.

## Are laptops made by Apple more expensive than those by other manufacturers?

In [62]:
manufacturer_price={}
for i in laptops_cleaned["manufacturer"].unique():
    lap = laptops_cleaned["manufacturer"] == i
    price = laptops_cleaned.loc[lap,"price_euros"]
    mean_price = price.mean()
    manufacturer_price[i] = mean_price
pd.Series(manufacturer_price)

{'Apple': 1564.1985714285713, 'HP': 1067.774854014598, 'Acer': 626.7758252427185, 'Asus': 1104.1693670886075, 'Dell': 1186.0689898989892, 'Lenovo': 1086.3844444444444, 'Chuwi': 314.2966666666667, 'MSI': 1728.9081481481485, 'Microsoft': 1612.3083333333334, 'Toshiba': 1267.8125, 'Huawei': 1424.0, 'Xiaomi': 1133.4625, 'Vero': 217.425, 'Razer': 3346.1428571428573, 'Mediacom': 295.0, 'Samsung': 1413.4444444444443, 'Google': 1677.6666666666667, 'Fujitsu': 729.0, 'LG': 2099.0}


Acer          626.775825
Apple        1564.198571
Asus         1104.169367
Chuwi         314.296667
Dell         1186.068990
Fujitsu       729.000000
Google       1677.666667
HP           1067.774854
Huawei       1424.000000
LG           2099.000000
Lenovo       1086.384444
MSI          1728.908148
Mediacom      295.000000
Microsoft    1612.308333
Razer        3346.142857
Samsung      1413.444444
Toshiba      1267.812500
Vero          217.425000
Xiaomi       1133.462500
dtype: float64

## No the laptops manufactured by Apple are not the most expensive.Instead Razer,LG,MSI,Microsoft,Google are more expensive than apple

## What is the best value laptop with a screen size of 15" or more?

In [70]:
filter_size = laptops_cleaned["screen_size_inches"] >= 15
filter_size = laptops.loc[b,["manufacturer","screen_size_inches","screen_resolution","cpu_manufacturer","cpu_speed_ghz","ram_GB","storage_1_type","storage_1_capacity_gb","storage_2_type","storage_2_capacity_gb","gpu_manufacturer","os","os_version","weight_kg","price_euros"]]
filter_ssd = (filter_size["storage_1_type"] == "SSD") & (filter_size["storage_2_type"] == "SSD")
filter_size.loc[filter_ssd]

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  
C:\Users\ASUS\Anaconda3\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,manufacturer,screen_size_inches,screen_resolution,cpu_manufacturer,cpu_speed_ghz,ram_GB,storage_1_type,storage_1_capacity_gb,storage_2_type,storage_2_capacity_gb,gpu_manufacturer,os,os_version,weight_kg,price_euros
28,Dell,15.6,1920x1080,Intel,1.9,NaN,SSD,256.0,SSD,256.0,Intel,Windows,10,1.88,1298.00
347,Dell,15.6,1920x1080,Intel,1.9,NaN,SSD,512.0,SSD,256.0,Intel,Windows,10,1.88,1607.96
901,Asus,17.3,1920x1080,Intel,2.7,NaN,SSD,256.0,SSD,256.0,Nvidia,Windows,10,3.58,1279.00


## For a bigger screen size,"ASUS" is the clear winner while the next one is "DELL"

## Which laptop has the most storage space?

In [71]:
filter_storage = (laptops_cleaned["storage_2_capacity_gb"] == 2000) & (laptops_cleaned["storage_1_capacity_gb"] == 512)
filter_manf = laptops_cleaned.loc[filter_storage,["manufacturer","storage_1_capacity_gb","storage_2_capacity_gb","price_euros"]]

In [72]:
filter_manf

,manufacturer,storage_1_capacity_gb,storage_2_capacity_gb,price_euros
370,Asus,512.0,2000.0,1099.0
894,MSI,512.0,2000.0,2649.0
977,Asus,512.0,2000.0,1799.0


## ASUS,MSI are clear winners here